In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import random
import matplotlib.pyplot as plt
import  networkx as nx
import tensorflow as tf

%matplotlib inline

In [54]:
def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=1.0, p_out=0.0):
    #gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out)
    A = nx.adjacency_matrix(G).todense()
    
    return A


communities = 2 #number of communities, chance to 
group_size = 6 #number of nodes in each communitites (balanced so far)
dim_graph = communities*group_size
A = np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.2)).astype(np.double)

data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.2)).astype(np.float64) for i in range(100)]
data[1]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]])

In [47]:
communities = 6
Adj = tf.placeholder(tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

r =  tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=2.0, dtype=tf.float64,
                                 seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([communities], dtype=tf.float64),
                                      tf.ones([communities], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([communities], dtype=tf.float64),
                                      tf.zeros([communities], dtype=tf.float64)])


y_a = tf.pack([true_assignment_a, true_assignment_b]) 
y_b = tf.pack([true_assignment_b, true_assignment_a]) 


eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])

centers = tf.Variable(tf.random_normal(shape=[2,2],
                                       mean=0.0, stddev=1.0, dtype=tf.float64,seed=None, name=None))

expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
expanded_centers = tf.expand_dims(centers, 1)

distances = tf.transpose(tf.reduce_sum(tf.square(tf.sub(expanded_vectors,
                                                        expanded_centers)), 2))

assignments = tf.nn.softmax(distances)

loss_vec_a = y_a*tf.log(tf.transpose(assignments))
loss_vec_b = y_b*tf.log(tf.transpose(assignments))

loss = tf.minimum(tf.reduce_sum(loss_vec_a), tf.reduce_sum(loss_vec_a))



optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r, centers])

with tf.Session() as sess:
    for i in range(10):
        sess.run(init)
#       sess.run(train)
        print sess.run([r, loss, centers], feed_dict = {Adj: data[i]})
    
    





[array([ 3.12317412]), -8.2351466971933238, array([[ 1.39119968, -0.26005339],
       [-0.24372163, -1.53984271]])]
[array([ 0.84121812]), -29.771346418053529, array([[-0.78583882,  2.88734801],
       [ 1.61611611, -0.45196944]])]
[array([-4.27378461]), -11.757282853953495, array([[-0.17404488,  0.27570597],
       [ 0.62918978, -0.99172488]])]
[array([-1.46065015]), -9.1523781351169085, array([[-0.52942376,  0.03586426],
       [ 0.29939199, -0.62895713]])]
[array([-1.02343599]), -8.5630697672028493, array([[ 0.08031587,  1.20777023],
       [ 1.12336889,  0.35355371]])]
[array([ 1.31698605]), -13.943264537005749, array([[ 0.64178432,  0.03893587],
       [-1.08966633, -0.23000412]])]
[array([-1.35852283]), -9.5560245273221334, array([[ 0.94398497,  0.55968787],
       [-0.61698521, -0.91500724]])]
[array([-0.68386662]), -11.317009505514989, array([[-0.0321034 ,  0.02024101],
       [ 1.22038767,  0.01220293]])]
[array([ 0.75248161]), -10.474147156943257, array([[-1.5840353 , -0.2075

In [4]:
#now with training uncommentedout

In [39]:
communities = 6
Adj = tf.placeholder(tf.float64)
Adj = tf.cast(Adj, tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

#r =  tf.Variable(tf.constant(2.1), dtype = tf.float64)
r = tf.Variable(tf.random_normal(shape=[1], mean=0.0, stddev=2.0, dtype=tf.float64,seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([communities], dtype=tf.float64),
                                      tf.ones([communities], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([communities], dtype=tf.float64),
                                      tf.zeros([communities], dtype=tf.float64)])

y_a = tf.pack([true_assignment_a, true_assignment_b]) 

eigenvec_proj = tf.nn.softmax(tf.slice(eigenvec, [0,0], [dim_graph, 2]))
eigenvec_proj = tf.cast(tf.reduce_sum(eigenvec, 1), dtype = tf.float64)


loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))), 
              tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))
#tf.reduce_sum(tf.square(tf.sub(c, true_assignment_a)))


optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(loss, var_list=[r])

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r])

with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        sess.run(train, feed_dict = {Adj: data[i]})
        print sess.run([r, loss_grad, eigenvec_grad, loss, y_a], feed_dict = {Adj: data[i]})
    
    



[array([-0.80005489]), [array([ 9.79694719])], [array([-1.98701186])], 80.859714974926163, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([-0.43659751]), [array([-4.58307354])], [array([ 1.68392781])], 77.553692330113861, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([-0.4539848]), [array([ 0.31788765])], [array([-1.11917682])], 81.061766717813185, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([ 0.12698334]), [array([ 11.42162481])], [array([-8.62615963])], 79.439033997694281, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([-0.3181855]), [array([ 6.73951226])], [array([ 5.23449881])], 82.768

In [33]:

tf.reset_default_graph()

In [38]:
?tf.pack


In [8]:
tw = tf.train.SummaryWriter('./logDir',sess.graph)

# Train your network

tw.flush()
tw.close()

In [64]:
communities = 2 #number of communities, chance to 
group_size = 6
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.8, p_out=0.5)).astype(np.float64) for i in range(1000)]
data[1]

array([[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.],
       [ 1.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.],
       [ 0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  0.]])

In [117]:
communities = 2
group_size=6
Adj = tf.placeholder(tf.float64)
Adj = tf.cast(Adj, tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

#r =  tf.Variable(tf.constant(2.1), dtype = tf.float64)
r = tf.Variable(tf.random_normal(shape=[1], mean=0.0, stddev=2.0, dtype=tf.float64,seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([group_size], dtype=tf.float64),
                                      tf.zeros([group_size], dtype=tf.float64)])

y_a = tf.pack([true_assignment_a, true_assignment_b]) 
y_b = tf.pack([true_assignment_b, true_assignment_a]) 


eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])


center = tf.Variable(tf.truncated_normal(shape=[2,2],
                                       mean=0.0, stddev=0.6, dtype=tf.float64,seed=None, name=None))

expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
expanded_centers = tf.expand_dims(center, 1)

distances = tf.reduce_sum(tf.square(tf.sub(expanded_vectors, expanded_centers)), 2)

hard_assignment = tf.argmin(distances, 0)
new_centers = tf.concat(0, [tf.reduce_mean(tf.gather(eigenvec_proj,
                                                     tf.reshape(tf.where(tf.equal(hard_assignment, c)),
                                                                [1,-1])), reduction_indices=[1]) for c in xrange(communities)])

#a = tf.slice(new_centers, [0,0], [1,2])
#b = tf.slice(new_centers, [1,0], [1,2])
update_centers = tf.assign(center, new_centers)


assignments = 1-tf.nn.softmax(tf.transpose(distances)) #this ensures that the smaller distances are actually the correct assignments as being far from the centers make it less likely it is in that cluster
#warning, the above transform will definitely not work for more than 2 centers, so need to be more clever later...
#like basically sum the other two entries and subtract, this will be the generalized algo.... 


loss_vec_a = y_a*tf.log(tf.transpose(assignments))
loss_vec_b = y_b*tf.log(tf.transpose(assignments))

loss = tf.minimum(-tf.reduce_sum(loss_vec_a), -tf.reduce_sum(loss_vec_a))


#loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))),tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))


optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(loss, var_list=[r])

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r, center])

with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        sess.run([train], feed_dict = {Adj: data[i]})
        if i%10 ==0:
            print sess.run([r, loss, center, hard_assignment, new_centers], feed_dict = {Adj: data[i]})
    
#, distances, assignments,  loss_grad, eigenvec_grad, 



[array([ 2.1520866]), 6.8865252187390915, array([[ 0.41104083, -0.20884815],
       [-0.84761614,  0.58082195]]), array([0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0]), array([[-0.3010572 , -0.18876819],
       [-0.26707646,  0.29913901]])]
[array([ 2.19829533]), 12.574520516228251, array([[ 0.41104083, -0.20884815],
       [-0.84761614,  0.58082195]]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([[ 0.28818551, -0.00571234],
       [        nan,         nan]])]
[array([ 2.14790093]), 10.843691687408963, array([[ 0.41104083, -0.20884815],
       [-0.84761614,  0.58082195]]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([[ 0.28490333, -0.01974335],
       [        nan,         nan]])]
[array([ 2.06278577]), 10.085500112919323, array([[ 0.41104083, -0.20884815],
       [-0.84761614,  0.58082195]]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([[ 0.28560289,  0.02171678],
       [        nan,         nan]])]
[array([ 2.00566133]), 10.967516492840904, array([[ 0.41104083, -0.208848

In [118]:

tf.reset_default_graph()

In [110]:
#does backprop not play well with update centers? 



In [119]:


center_1 = tf.Variable(tf.random_normal(shape=[2,1],
                                       mean=1.0, stddev=1.0, dtype=tf.float64,seed=None, name=None))
center_2 = tf.Variable(tf.random_normal(shape=[2,1],
                                       mean=-1.0, stddev=1.0, dtype=tf.float64,seed=None, name=None))
center = tf.pack([center_1, center_2])

In [ ]:

#let's try another loss: 


communities = 2
group_size=6
Adj = tf.placeholder(tf.float64)
Adj = tf.cast(Adj, tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

#r =  tf.Variable(tf.constant(2.1), dtype = tf.float64)
r = tf.Variable(tf.random_normal(shape=[1], mean=0.0, stddev=2.0, dtype=tf.float64,seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([group_size], dtype=tf.float64),
                                      tf.zeros([group_size], dtype=tf.float64)])

y_a = tf.pack([true_assignment_a, true_assignment_b]) 
y_b = tf.pack([true_assignment_b, true_assignment_a]) 


eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])


center = tf.Variable(tf.truncated_normal(shape=[2,2],
                                       mean=0.0, stddev=0.6, dtype=tf.float64,seed=None, name=None))

expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
expanded_centers = tf.expand_dims(center, 1)

distances = tf.reduce_sum(tf.square(tf.sub(expanded_vectors, expanded_centers)), 2)

hard_assignment = tf.argmin(distances, 0)
new_centers = tf.concat(0, [tf.reduce_mean(tf.gather(eigenvec_proj,
                                                     tf.reshape(tf.where(tf.equal(hard_assignment, c)),
                                                                [1,-1])), reduction_indices=[1]) for c in xrange(communities)])

#a = tf.slice(new_centers, [0,0], [1,2])
#b = tf.slice(new_centers, [1,0], [1,2])
update_centers = tf.assign(center, new_centers)


assignments = 1-tf.nn.softmax(tf.transpose(distances)) #this ensures that the smaller distances are actually the correct assignments as being far from the centers make it less likely it is in that cluster
#warning, the above transform will definitely not work for more than 2 centers, so need to be more clever later...
#like basically sum the other two entries and subtract, this will be the generalized algo.... 


loss_vec_a = y_a*tf.log(tf.transpose(assignments))
loss_vec_b = y_b*tf.log(tf.transpose(assignments))

loss = tf.minimum(-tf.reduce_sum(loss_vec_a), -tf.reduce_sum(loss_vec_a))


#loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))),tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))


optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(loss, var_list=[r])

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r, center])

with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        sess.run([train], feed_dict = {Adj: data[i]})
        if i%10 ==0:
            print sess.run([r, loss, center, hard_assignment, new_centers], feed_dict = {Adj: data[i]})
    
#, distances, assignments,  loss_grad, eigenvec_grad, 

